In [8]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating environment at `~/code/sanna21/Project.toml`


$\newcommand{\C}[]{\mathbb{C}}$

In [9]:
using HomotopyContinuation, LinearAlgebra
set_default_compile(:none)

:none

# Lecture: Parameter Homotopies

In this lecture we focus on a general framework for the computation of *isolated solutions* of polynomial system with the homotopy continuation method.

### The Basic Idea

The basic idea for a given polynomial system $F$ that we want to solve is the following.

* Put the polynomial system $F$ into a *family* of polynomial systems $\mathcal{F}_Q$ depending on a parameter set $Q$. Then there exists a $p \in Q$ such that $F = F_p  \in \mathcal{F}_Q$.
* Solve a general system $F_q \in \mathcal{F}_Q$.
* Deform the start system $F_q$ to the target system $F_p$ by moving **inside the family $\mathcal{F}_Q$**along a path $\gamma: [0,1] \rightarrow Q$ with $\gamma(1) = q$ and $\gamma(0)= p$ and track the solutions of $F_q$ as it is deformed to $F_p$.


In the last step, we constructed the *homotopy* $H(x,t) = F_{\gamma(t)}(x)$ and the problem of tracking a solution is a *continuation* problem giving the method its name.

### What is a family of polynomial systems?
For us a family of polynomial systems is simply a parameterized polynomial system $F(x;p): \mathbb{C}^n \times Q \rightarrow \mathbb{C}^N$ where $Q \subseteq \mathbb{C}^m$ is an irreducible algebraic variety.
For $p \in Q$, we write $F_p \in \mathcal{F}_Q$ for the polynomial system with fixed parameter values $p$.

**Example:**

Given $F(x; a, b) = x^2 + ax + b$ we consider the family $\mathcal{F}_{\C^2} = \{x^2 + ax + b\,| \,a, b \in \C \}$. This is the family of quadratic polynomials.

In [10]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

System of length 1
 1 variables: x
 2 parameters: a, b

 b + a*x + x^2

For $(2,3) \in \C^2$ we have $F_{(2,3)} = x^2 + 2x + 3 \in \mathcal{F}_{\C^2}$.

## The incidence variety
Consider

$$
Z = \{ (x, p) \in \mathbb{C}^n \times Q \, | \, F(x;p) = 0\} \subseteq \mathbb{C}^n \times Q
$$

We denote by $\pi_1: Z \rightarrow \mathbb{C}^n, \; (x,p) \mapsto x$ and  $\pi_2: Z \rightarrow Q, \; (x,p) \mapsto p$ the projections onto the first and second factor.


**Example: Quadratic Polynomials**

$$
Z = \{ (x; a, b) \in \mathbb{C} \times \mathbb{C}^2\, | \,  x^2 + ax + b = 0\} \subseteq \mathbb{C} \times \mathbb{C}^2
$$

The following graphic displays how a change of parameters (right hand side) results in a change of the solutions.
The right figure also displays the curve $a^2-4b$, which is the discrimant of $F$. The descriminant describes the parameter values $(a,b)$ where our quadratic polynomial has a double root.

![roots](roots_quadratic_12fps_200dpi.gif)

**Oberservation:**
The solutions move continuously with a change in the paramter values as long as the parameter value does not cross the discriminant.

## Path tracking


Let $H(x, t)$ : $\C^n \times \C \rightarrow \C^N$ be a polynomial homotopy.
The adjective polynomial refers to the property of $H(x, t) $ that it is a polynomial system for all $t \in \C$.
Additionally, we require that $H$ is holomorphic in $t$.

We assume to have a solution $x_1$ to $H(x,1) = 0$ and we want to compute
a solution of $H(x,0) = 0$.
For this, we consider the solution path $x(t)$ implicitly defined by the conditions

\begin{equation}\label{eq:path_implicit}
    H(x(t),t)=0 \textrm{ for all } t \in (0,1] \textrm{ and } x(1)=x_1.
\end{equation}

From this condition follows that the solution path $x(t)$ is also the solution to the Davidenko differential equation

\begin{equation}\label{eq:path_ode}
    H_x(x(t),t) \dot{x}(t) + H_t(x(t),t) = 0
\end{equation}

with initial value $x(0)=x_0$ where $H_x$ and $H_t$ denote the partial derivatives with respect to $x$ and $t$.

We say that a solution path $x(t)$ is *smooth* (or *nonsingular*) if there exists a continuous path $x: (0, 1] \rightarrow \C^N$ such that $x(1) = x_1$, $H(x(1), 1) = 0$ and for all $t \in (0, 1]$ the point $x(t)$ is a **regular** isolated solution of $H(x,t)$.


## Parameter Homotopy

Let's assume we have $p, q \in Q$ and a path $\gamma: [0,1] \rightarrow Q$ with $\gamma(1) = q$ and $\gamma(0)= p$.
Additionaly, we have a regular isolated solution $s \in \C^n$ of $F_q$.
The parameter homotopy
$$H(x,t) = F(x; \gamma(t))$$
together with our solution $s$ defines a solution path $x(t)$ that we can track.

**Example**

Recall our quadratic polynomials
![roots](roots_quadratic_12fps_200dpi.gif)

Let's track the solutions of $x^2 - 1$ ( $-1$ and $1$) to $x^2 + 1.5x - 2$.

We have $q = [0, 1]$ and $ p = [1.5, -2]$ and we use $\gamma(t) = t q + (1 - t) p$.

In [11]:
@var t
q = [0, -1]
p = [1.5, -2]
H = Homotopy(F([x], q * t + (1-t)*p), [x], t)

Homotopy in t of length 1
 1 variables: x

 -t + 1.5*x*(1 - t) - 2.0*(1 - t) + x^2

In [12]:
# Construct a tracker object to track
tracker = Tracker(H)

Tracker{InterpretedHomotopy{Float64, Float64}, Matrix{ComplexF64}}()

In [13]:
r₁ = track(tracker, [1], 1, 0)

TrackerResult:
 • return_code → success
 • solution → ComplexF64[0.8507810593582122 + 0.0im]
 • t → 0.0 + 0.0im
 • accuracy → 0.0
 • accepted_steps → 3
 • rejected_steps → 0
 • extended_precision → false
 • extended_precision_used → false
 • ω → 0.28635
 • μ → 2.2204e-16
 • τ → 2.4118


In [14]:
r₂ = track(tracker, [-1], 1, 0)

TrackerResult:
 • return_code → success
 • solution → ComplexF64[-2.350781059358212 + 0.0im]
 • t → 0.0 + 0.0im
 • accuracy → 3.3598e-17
 • accepted_steps → 3
 • rejected_steps → 0
 • extended_precision → false
 • extended_precision_used → false
 • ω → 0.56057
 • μ → 2.2204e-16
 • τ → 2.4118


Let's see whether we got what we wanted:

In [15]:
expand((x - real(solution(r₁)[1])) * (x - real(solution(r₂)[1])))

-2.0 + 1.5*x + x^2

**Crossing the discriminant**

What happens when we cross the discriminant?

In [16]:
@var t
q = [0, -1]
p = [1, 1]
H = Homotopy(F([x], q * t + (1-t)*p), [x], t)

Homotopy in t of length 1
 1 variables: x

 1 - 2*t + x*(1 - t) + x^2

In [17]:
track(Tracker(H), [1], 1, 0)

TrackerResult:
 • return_code → terminated_step_size_too_small
 • solution → ComplexF64[-0.2679491828696726 + 0.0im]
 • t → 0.4641016151377546 + 0.0im
 • accuracy → 0.0
 • accepted_steps → 24
 • rejected_steps → 0
 • extended_precision → false
 • extended_precision_used → false
 • ω → 16855.0
 • μ → 2.2204e-16
 • τ → 1.0556e-16


Since the path is no more smooth, the tracking fails! We can also look at each of the steps:

In [18]:
for (x, t) in iterator(Tracker(H), [1])
    println("t = ", t, "\nx = ", x[1], "\n")
end

t = 1.0
x = 1.0 + 0.0im

t = 0.8176424407141405
x = 0.7110670999260805 + 0.0im

t = 0.5479059376109063
x = 0.15724024427084235 + 0.0im

t = 0.48165428917735986
x = -0.08459007465452066 + 0.0im

t = 0.4676475464945976
x = -0.18778696296294087 + 0.0im

t = 0.46481225180210795
x = -0.2325084774288432 + 0.0im

t = 0.4642438007715584
x = -0.25218485915715627 + 0.0im

t = 0.46413005459884926
x = -0.2609165117755144 + 0.0im

t = 0.4641073031233653
x = -0.2648075766787253 + 0.0im

t = 0.4641027527386125
x = -0.2665449203827741 + 0.0im

t = 0.4641018426580756
x = -0.2673213233295415 + 0.0im

t = 0.46410166064182473
x = -0.2676684289410933 + 0.0im

t = 0.4641016242385689
x = -0.2678236368042547 + 0.0im

t = 0.46410161695791746
x = -0.26789304337242054 + 0.0im

t = 0.46410161550178713
x = -0.26792408203433643 + 0.0im

t = 0.4641016152105611
x = -0.2679379627659967 + 0.0im

t = 0.4641016151523159
x = -0.26794417037708335 + 0.0im

t = 0.46410161514066683
x = -0.2679469465044944 + 0.0im

t = 0.46410

We will learn how to avoid these kinds of failures in a moment. But before we need to do some theory.

## The Parameter Continuation  Theorem [[Morgan, Sommese '89](https://www.sciencedirect.com/science/article/abs/pii/0096300389900994)]

Let $Q \subseteq \C^m$ be an irreducible algebraic variety and let $F(x;p): \C^n \times Q \rightarrow \C^N$ be a system of $N$ polynomials in $n$ variables, $N \ge n$, with $m$ parameters.

Given $p \in Q$, we write $F_p(x) = F(x; p)$.
Consider the incidence variety

\begin{equation*}
    Z = \{ (x, p) \in \C^n \times Q \, | \, F(x;p) = 0\} \subseteq \C^n \times Q
\end{equation*}

and let $Y \subset Z$ be a top-dimensional irreducible component of $Z$.


We denote by $\pi_1: Y \rightarrow \C^n, \; (x,p) \mapsto x$ and  $\pi_2: Y \rightarrow Q, \; (x,p) \mapsto p$ the projections onto the first and second factor.

Furthermore, let $\mathcal{N}(p, Q)$ denote the number of regular isolated solutions of $F_p$ contained in $\pi_1 (\pi_2^{-1}(p))$ as a function of $p \in Q$.

Then
* $\mathcal{N}(p, Q)$ is finite, and there exists a Zariski open set $U \subset Q$ such that $\mathcal{N}(p, Q)$ is the same, say $\mathcal{N}(Q)$, for all $p \in U$. We denote the exceptional set by $\Sigma = Q \setminus U$.

* The homotopy $H(x,t)=F_{\gamma(t)}(x)$ with $\gamma(t): [0, 1] \rightarrow U$ has $\mathcal{N}(Q)$ continuous, isolated smooth solution paths $x(t) \in \pi_1(\pi_2^{-1}(\gamma(t)))$;

* As $t \rightarrow 0$, the limits of the solution paths, if they exist, of the homotopy $F_{\gamma(t)}(x)$ with $\gamma(t): [0, 1] \rightarrow Q$ and $\gamma(t) \in U$ for $t \in (0, 1]$ include all the regular isolated solutions of $F_{\gamma(0)}$ contained in $\pi_1(\pi_2^{-1}(\gamma(0)))$.

  Additionally, if for all $p \in U$ the set $\pi_1(\pi_2^{-1}(p))$ has cardinality $\mathcal{N}(Q)$, then the limits of the solution paths include all isolated solutions of $F_{\gamma(0)}$ contained in $\pi_1(\pi_2^{-1}(\gamma(0)))$. This includes the isolated solutions with multiplicity greater than one.

**Takeaway**

If for a general $q \in Q$ the system $F_q$ has only regular isolated solutions and we know all of them, then we can compute for almost all $p\in Q$ all isolated solutions of $F_p$ by using the homotopy $H(x,t) = F_{\gamma(t)}(x)$ for a general path $\gamma(t): [0,1] \rightarrow Q$ with $\gamma(1) = q$ and $\gamma(0) = p$.

**Example**

We consider again our quadratic polynomials.

We have: $Q = \C^2$ and $\mathcal{N}(\C^2) = 2$. The expectional set is $\Sigma = \{(a,b) \in \C^2 \, | \, a^2  - 4b = 0\}$.

## How to avoid the discriminant

In our previous example the path tracking failed since our path $\gamma(t)$ in parameter space hit the discriminant

We can avoid hitting the discriminant by using the following observation:

* The exceptional set has (complex) codimension 1 resp. real codimension 2. Our path $\gamma(t)$ for $t \in [0,1]$ is of real dimension 1.
* If $Q = \C^m$, then for almost all $q\ \in \C^m$ the line segment $\{ t q + (1 - t)p \, | \, t \in (0,1] \}$ does *not* meet the exceptional set.

Therefore, if $F(x;p)$ is a parameterized polynomial system with $\C^m$ as the parameter space, then the homotopy

\begin{equation}
    H(x,t) = F(x; t q + (1 - t)p)
\end{equation}

with general $q \in \C^m$ has smooth solution paths whose endpoints, for $t$ from 1 to 0, include all isolated zeros of $F_p$.

**Example (continued)**

Choose general start parameters to avoid the discriminant!

In [19]:
x₁, x₂ = randn(ComplexF64, 2)

2-element Vector{ComplexF64}:
    1.9151077142346105 + 0.5782379740278883im
 -0.009098760416020191 + 0.6189687397182608im

In [20]:
q = [-(x₁ + x₂), x₁ * x₂]

2-element Vector{ComplexF64}:
 -1.9060089538185903 - 1.1972067137461493im
 -0.3753363363039753 + 1.1801305595153915im

In [21]:
F([x₂], q)

1-element Vector{ComplexF64}:
 5.551115123125783e-17 + 1.3704315460216776e-16im

In [22]:
# HC has a built-in parameter homotopy 
H = ParameterHomotopy(F; start_parameters=q, target_parameters=[1, 1])
tracker = Tracker(H)
track(tracker, [x₂])

TrackerResult:
 • return_code → success
 • solution → ComplexF64[-0.5 + 0.8660254037844387im]
 • t → 0.0 + 0.0im
 • accuracy → 6.4126e-17
 • accepted_steps → 7
 • rejected_steps → 0
 • extended_precision → false
 • extended_precision_used → false
 • ω → 1.1542
 • μ → 2.2204e-16
 • τ → 0.84249


## The offline-online approach

Our previous discussion gives us the following approach for repeatedly solving a parameterized polynomial system $F(x;p)$ with parameter space $\C^m$.

1. Compute all isolated solutions $S_q$ for a general $q \in \C^m$ by some method (offline part - needs to be done only once)
2. For each parameter value $p \in \C^m$ of interest
    1. Construct the homotopy $H(x, t) = F(x; t q + (1-t) p)$
    2. For each $s \in S_q$ track the solution from $t=1$ to $t=0$. The result is $S_p$.

Offline part:

In [23]:
x₁, x₂ = randn(ComplexF64, 2)
q = [-(x₁ + x₂), x₁ * x₂]
S_q = [[x₁], [x₂]]

2-element Vector{Vector{ComplexF64}}:
 [-0.45597188199260896 - 0.11476557636376941im]
 [1.4964537977795396 + 0.06931332612082547im]

In [24]:
write_parameters("quad_start_params.txt", q)
write_solutions("quad_start_sols.txt", S_q)

Online part:

In [25]:
q = read_parameters("quad_start_params.txt")

2-element Vector{ComplexF64}:
 -1.0404819157869305 + 0.04545225024294394im
 -0.6743860706665773 - 0.20334631036240075im

In [26]:
S_q = read_solutions("quad_start_sols.txt")

2-element Vector{Vector{ComplexF64}}:
 [-0.45597188199260896 - 0.11476557636376941im]
 [1.4964537977795396 + 0.06931332612082547im]

In [27]:
p = [-2.5, 3]
R_p = solve(F, S_q; start_parameters = q, target_parameters = p)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (0 real)
• random_seed: 0x12d73766


In [28]:
S_p = solutions(R_p)

2-element Vector{Vector{ComplexF64}}:
 [1.25 - 1.1989578808281798im]
 [1.25 + 1.1989578808281798im]

## An Example from Computer Vision

We follow https://www.juliahomotopycontinuation.org/examples/computer-vision/

We want to reconstruct a three-dimensional object from two-dimensional images. Think of an object, maybe a [dinosaur], that is captured by two cameras from different angles. Suppose we have the following information available:

- the configuration of the cameras (in the form of [camera](https://en.wikipedia.org/wiki/Pinhole_camera_model) [matrices](https://en.wikipedia.org/wiki/Camera_matrix);
- sets of 2D points in the two 2D images, where each set corresponds to one 3D point.

(Actually, the second bullet is the more challenging part in reconstructing three-dimensional objects. We assume that this task has already been done.)

The goal is to compute (part of) the following 3D picture

<p style="text-align:center;"><img src="dino.png" width="30%"/></p>

In theory, it suffices to have two matching 2D points and the configuration of the cameras to *exactly* reconstruct the corresponding 3D point. In practice, there is lens distortion, pixelation, the algorithm matching the 2D points is not working fully precise and so on. This is why we are looking for 3D points that minimize the euclidean distance regarding their 2D image points. We want to solve an optimization problem.

In [29]:
A₁, A₂ = include("cameras.txt");

In [30]:
A₁

3×4 Matrix{Float64}:
   3.99236  39.4177     -0.76329     3.95918
 -14.4302   -0.941442  -27.451     -14.4294
   7.93751  -0.094446   -0.368913    7.93759

In [31]:
A₂

3×4 Matrix{Float64}:
  10.7732   38.1265    -0.76329     3.95918
 -14.3746    1.57741  -27.451     -14.4294
   7.80064  -1.47067   -0.368913    7.93759

In [32]:
camera_points = include("camera_points.txt")

257×4 Matrix{Float64}:
 0.625926  0.0313426  0.632269  0.0377006
 0.667515  0.0750772  0.676019  0.0840586
 0.618364  0.105      0.619907  0.110432
 0.569877  0.114012   0.566713  0.115525
 0.636775  0.124475   0.645093  0.131651
 0.537068  0.135525   0.532577  0.134769
 0.541667  0.134306   0.537114  0.13392
 0.496605  0.140309   0.497284  0.136667
 0.515355  0.141188   0.511173  0.138966
 0.552654  0.140448   0.547994  0.140648
 0.467006  0.171327   0.463256  0.167654
 0.5325    0.169892   0.526265  0.16929
 0.483318  0.178534   0.479568  0.174228
 ⋮                              
 0.359568  0.600617   0.345741  0.584722
 0.455772  0.570386   0.441373  0.562469
 0.416605  0.60037    0.401312  0.589398
 0.530926  0.592361   0.515787  0.591481
 0.529228  0.443796   0.509861  0.443349
 0.684846  0.394846   0.671451  0.405664
 0.285201  0.658657   0.264475  0.635494
 0.298981  0.673102   0.277747  0.651096
 0.427701  0.645602   0.411821  0.635154
 0.363812  0.656065   0.347377  0.63946
 0

Let $x\in\mathbb{R}^3$ be a 3D point. Then, taking a picture $u=(u_1,u_2)\in\mathbb{R}^2$ of $x$ is modeled as
$$\begin{pmatrix} u_1 \\\ u_2 \\\ 1 \end{pmatrix} = t  A  \begin{pmatrix}x_1 \\\ x_2 \\\ x_3  \\\ 1\end{pmatrix},$$

where $A\in \mathbb{R}^{3\times 4}$ is the camera matrix and $t \in \mathbb{R}, t \neq 0$, is a scalar. This is called the pinhole camera model. Let us write $y(x)$ for the first two entries of  $A \begin{pmatrix} x\\\ 1\end{pmatrix}$, and $z(x)$ for the third entry. Then, $u=t y(x)$ and $1 = t z(x)$.

If $p = (p_1, p_2) \in\mathbb{R}^2\times \mathbb{R}^2$ are two input pictures from the data set, we want to solve the following minimization problem:

$$
\underset{(x,t) \in \mathbb{R}^3\times (\mathbb{R}\backslash \\{0\\})^2}{\operatorname{argmin}} \lVert t_1  y_1(x) - p_1 \rVert^2 + \lVert  t_2   y_2(x) - p_2 \rVert^2
$$

$$
\text{s.t. } t_1  z_1(x) = 1 \text{ and } t_2  z_2(x)=1.\quad $$


In [54]:
# declare variables
@var u₁[1:2] u₂[1:2] x[1:3] t[1:2]

# objective
f = sum(x -> x^2, [
    t[1] * (A₁[1:2,:] * [x; 1]) - u₁
    t[2] * (A₂[1:2,:] * [x; 1]) - u₂
])

# constraints
G = [
    t[1] * (A₁[3,:] ⋅ [x; 1]) - 1
    t[2] * (A₂[3,:] ⋅ [x; 1]) - 1
]

# construct lagrange multiplier
@var λ[1:2]
L = f - λ ⋅ G
∇L = differentiate(L, [x;t;λ])
crit_sys = System(∇L;
        variables = [x;t;λ],
        parameters = [u₁; u₂])

System of length 7
 7 variables: x₁, x₂, x₃, t₁, t₂, λ₁, λ₂
 4 parameters: u₁₁, u₁₂, u₂₁, u₂₂

 -7.93750752*t₁*λ₁ + 7.98471376*t₁*(-u₁₁ + t₁*(3.95917551 + 3.99235688*x₁ + 39.41768098*x₂ - 0.76328988*x₃)) - 28.86046202*t₁*(-u₁₂ + t₁*(-14.42943344 - 14.43023101*x₁ - 0.94144158*x₂ - 27.45097011*x₃)) - 7.80064344*t₂*λ₂ + 21.54649822*t₂*(-u₂₁ + t₂*(3.95917551 + 10.77324911*x₁ + 38.12649461*x₂ - 0.76328988*x₃)) - 28.74923612*t₂*(-u₂₂ + t₂*(-14.42943344 - 14.37461806*x₁ + 1.57741398*x₂ - 27.45097011*x₃))
 0.094446*t₁*λ₁ + 78.83536196*t₁*(-u₁₁ + t₁*(3.95917551 + 3.99235688*x₁ + 39.41768098*x₂ - 0.76328988*x₃)) - 1.88288316*t₁*(-u₁₂ + t₁*(-14.42943344 - 14.43023101*x₁ - 0.94144158*x₂ - 27.45097011*x₃)) + 1.47067488*t₂*λ₂ + 76.25298922*t₂*(-u₂₁ + t₂*(3.95917551 + 10.77324911*x₁ + 38.12649461*x₂ - 0.76328988*x₃)) + 3.15482796*t₂*(-u₂₂ + t₂*(-14.42943344 - 14.37461806*x₁ + 1.57741398*x₂ - 27.45097011*x₃))
 0.36891288*t₁*λ₁ - 1.52657976*t₁*(-u₁₁ + t₁*(3.95917551 + 3.99235688*x₁ + 39.41768098*x₂ - 0

We compute for a general $q \in \C^4$ all isolated solutions.

In [55]:
q = randn(ComplexF64, 4)
R_q = solve(crit_sys, target_parameters = q)

Result with 6 solutions
• 32 paths tracked
• 6 non-singular solutions (0 real)
• random_seed: 0xa87fb8bc
• start_system: :polyhedral


In [56]:
S_q = solutions(R_q);

Now, we can use our start pair to solve for some of the observed points.

In [57]:
p = camera_points[1,:]
R_p = solve(crit_sys, S_q, start_parameters = q, target_parameters = p)

Result with 6 solutions
• 6 paths tracked
• 6 non-singular solutions (2 real)
• random_seed: 0xa3bdf640


We computed *all* critical points, so we need a small helper to find the global minimum.

In [58]:
function reconstruct_point_from_crits(res, p)
    S = real_solutions(res)
    val, ind = findmin([f([x;t] => s[1:5], [u₁; u₂] => p) for s in S])
    S[ind][1:3]
end

reconstruct_point_from_crits (generic function with 1 method)

In [59]:
reconstruct_point_from_crits(R_p, p)

3-element Vector{Float64}:
  0.008696731518828328
  0.0184951510725211
 -0.5402916914796865

Now let's solve for all camera points.

In [60]:
xs = solve(crit_sys, S_q,
        start_parameters = q,
        target_parameters = collect(eachrow(camera_points)),
        transform_result = reconstruct_point_from_crits)

257-element Vector{Vector{Float64}}:
 [0.008696731518828328, 0.0184951510725211, -0.5402916914796865]
 [0.01159622104147512, 0.02703648371277111, -0.5552160252548322]
 [0.002890146868773516, 0.01653284246934995, -0.5589617867909924]
 [-0.0032480732176274193, 0.006433036050043622, -0.5577833387815955]
 [0.011159429040447035, 0.020422987629404005, -0.5696059601997782]
 [-0.0053255835690869755, -0.00039963289681372617, -0.5627500392122724]
 [-0.005322668783377026, 0.0005493885038636538, -0.5624312491770072]
 [5.0785674124773326e-5, -0.008776518140924474, -0.5669081438465506]
 [-0.0053502572531626425, -0.004879442735028869, -0.5642641630332916]
 [-0.005271936160069156, 0.0027777013249991986, -0.5643282128970074]
 [-0.0059053946243271, -0.014920312244062879, -0.5728159404084235]
 [-0.007480687182210848, -0.0014758113248857572, -0.5717206979478144]
 [-0.005502954391835501, -0.01161583941929105, -0.5750354078530086]
 ⋮
 [-0.02085813325277794, -0.03842212566009838, -0.6891045518449951]
 [-0.01

Finally, we can visualize what we got out.

In [61]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [62]:
X = reduce(hcat, xs)
scatter(X[1,:], X[2,:], X[3,:], markersize = 2, markerstrokewidth = 1, legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

With more camera / point pairs we could recover the dino completely.